In [49]:
import matplotlib.pyplot as plt
import pandas as pd
from scipy.constants import precision
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay, confusion_matrix, recall_score, \
    classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import matplotlib.pyplot as plt

In [2]:
mbank = pd.read_csv(r"D:\DAIAugust2025\ML\250840128012_PML\BankMarketing.csv")
mbank.head()

age        job  marital    education  default housing loan    contact  \
0   56  housemaid  married     basic.4y       no      no   no  telephone   
1   57   services  married  high.school  unknown      no   no  telephone   
2   37   services  married  high.school       no     yes   no  telephone   
3   40     admin.  married     basic.6y       no      no   no  telephone   
4   56   services  married  high.school       no      no  yes  telephone   

  month day_of_week  ...  campaign  pdays  previous     poutcome emp.var.rate  \
0   may         mon  ...         1    999         0  nonexistent          1.1   
1   may         mon  ...         1    999         0  nonexistent          1.1   
2   may         mon  ...         1    999         0  nonexistent          1.1   
3   may         mon  ...         1    999         0  nonexistent          1.1   
4   may         mon  ...         1    999         0  nonexistent          1.1   

   cons.price.idx  cons.conf.idx  euribor3m  nr.employed   y  
0          93.994          -36.4      4.857       5191.0  no  
1          93.994          -36.4      4.857       5191.0  no  
2          93.994          -36.4      4.857       5191.0  no  
3          93.994          -36.4      4.857       5191.0  no  
4          93.994          -36.4      4.857       5191.0  no  

[5 rows x 21 columns]

In [4]:
X = mbank.drop('y', axis=1)
y = mbank['y']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=25, stratify=y)

In [3]:
mbank.isnull().sum()

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64

In [35]:
ss = StandardScaler()
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=True, drop='first')
trns = make_column_transformer((ohe, make_column_selector(dtype_include=object)),(ss, make_column_selector(dtype_exclude=object)), remainder='passthrough', verbose_feature_names_out=False)

rf = RandomForestClassifier()

In [36]:
pipe = Pipeline([('scl_enc', trns), ('model', rf)])
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

In [52]:
print(f"accuracy before gcv --> {accuracy_score(y_test, y_pred)}")
confusion_matrix(y_test, y_pred)

accuracy before gcv --> 0.9106579266812334


array([[7072,  238],
       [ 498,  430]])

In [51]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          no       0.93      0.97      0.95      7310
         yes       0.64      0.46      0.54       928

    accuracy                           0.91      8238
   macro avg       0.79      0.72      0.74      8238
weighted avg       0.90      0.91      0.90      8238



In [23]:
pipe.get_params()

{'memory': None,
 'steps': [('scl_enc',
   ColumnTransformer(remainder='passthrough',
                     transformers=[('onehotencoder',
                                    OneHotEncoder(drop='first',
                                                  handle_unknown='ignore'),
                                    <sklearn.compose._column_transformer.make_column_selector object at 0x000001AED22E1350>),
                                   ('standardscaler', StandardScaler(),
                                    <sklearn.compose._column_transformer.make_column_selector object at 0x000001AED2311B50>)],
                     verbose_feature_names_out=False)),
  ('model', RandomForestClassifier())],
 'transform_input': None,
 'verbose': False,
 'scl_enc': ColumnTransformer(remainder='passthrough',
                   transformers=[('onehotencoder',
                                  OneHotEncoder(drop='first',
                                                handle_unknown='ignore'),
             

In [53]:
params = {'model__n_estimators': [50,100], 'model__max_depth': [5, 8], 'model__min_samples_split': [2,3]}

gcv = GridSearchCV(estimator=pipe, param_grid=params, n_jobs=-1, cv=3)
gcv.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('scl_enc',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('onehotencoder',
                                                                         OneHotEncoder(drop='first',
                                                                                       handle_unknown='ignore'),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x000001AED36C7D70>),
                                                                        ('standardscaler',
                                                                         StandardScaler(),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x000001AED47C77D0>)],
                                                          verbose_feature_names_out=False)),
                                       ('model', RandomForestClassifier())]),
             n_jobs=-1,
             param_grid={'model__max_depth': [5, 8],
                         'model__min_samples_split': [2, 3],
                         'model__n_estimators': [50, 100]})

In [32]:
gcv.best_params_

{'model__max_depth': 8,
 'model__min_samples_split': 3,
 'model__n_estimators': 100}

In [55]:
y_pred = gcv.predict(X_test)
print(f"after gcv --->{accuracy_score(y_test, y_pred)}")

after gcv --->0.9049526584122359
